In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jyban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jyban\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False

In [4]:
stopwords = set(stopwords.words("english"))
data = pd.read_csv("CCAC_data.csv", dtype={"Body": str})
body_tokens = [[word.lower() for word in word_tokenize('' if isnan(s) else s) if not word in stopwords and word.isalnum()] for s in data['Body']]

import operator
d = {}
for w in body_tokens:
  for word in w:
    if word not in d:
      d[word] = 0
    d[word] = d[word] + 1
dict( sorted(d.items(), key=operator.itemgetter(1),reverse=True))

{'email': 38908,
 'https': 15761,
 'i': 14306,
 'mailto': 14140,
 'meeting': 6689,
 'brandon': 6499,
 '2021': 6037,
 '3a': 5888,
 'join': 5119,
 '2f': 5095,
 'to': 5087,
 'sent': 4633,
 'akcali': 4491,
 'subject': 4422,
 'from': 4388,
 'sales': 4365,
 'aristeo': 4239,
 '22': 4152,
 'ccac': 3955,
 'time': 3565,
 'please': 3490,
 'the': 3422,
 'http': 3349,
 're': 3235,
 'pm': 3157,
 'we': 3012,
 'us': 2991,
 'would': 2785,
 'data': 2546,
 'microsoft': 2505,
 'a': 2427,
 'if': 2410,
 'adams': 2334,
 'am': 2304,
 'this': 2261,
 'get': 2180,
 'like': 2131,
 'know': 2102,
 'ext': 2007,
 'thanks': 2003,
 'good': 2000,
 'cc': 1977,
 'call': 1966,
 'mobile': 1949,
 'team': 1893,
 'let': 1877,
 'teams': 1838,
 'conference': 1819,
 'video': 1817,
 'click': 1815,
 'number': 1795,
 'hi': 1786,
 'best': 1770,
 'need': 1712,
 'it': 1711,
 'information': 1697,
 'work': 1694,
 'great': 1694,
 'business': 1693,
 'help': 1634,
 'learn': 1588,
 'id': 1571,
 'alexander': 1545,
 '2': 1524,
 'm': 1522,
 'ne

In [9]:
phishing_terms = ["paypal", "login", "bank", "verify", "suspend", "won", "account", "$", "clicking" ,"job", "important", "reward" , "link", "visit", " click", "urgent", "offer", "http://", "activate", "click here", "visit here", "claim", "bonus", "expiry", "save", "hiring", "go here", "gift"]
good_words = ["microsoft", "teams", "meeting", "https", "sent", "reminder", "discussion", "objectives", "report", "tomorrow", "thank", "schedule", "promotional", "join", "conference", "video", "monday", "tuesday", "wednesday", "thursday", 'friday', "fireeye", "webex", "outlook"]
for p in phishing_terms:
  data[p + '_count'] = [0 if isnan(s) else s.lower().count(p) for s in data['Body']]
  data[p + '_scount'] = [0 if isnan(s) else s.lower().count(p) for s in data['Subject']]

for p in good_words:
  data[p + '_count'] = [0 if isnan(s) else s.lower().count(p) for s in data['Body']]
    
data['Microsoft Teams Meeting_count'] = [0 if isnan(s) else s.count('Microsoft Teams Meeting') for s in data['Body']]
data['reply_thread_count'] = [0 if isnan(s) else min(s.count('From:'), s.count('Sent:'), s.count('To:')) for s in data['Body']]
data['https://'] = [0 if isnan(s) else s.count('https://') for s in data['Body']]
data['EmailTo_isnan'] = [1 if isnan(e) else 0 for e in data['EmailTo']]
data['Body_isnan'] = [1 if isnan(e) else 0 for e in data['Body']]


In [11]:
values = []
sources = []
for index, row in data.iterrows():
  val = 0
  source = 0
  for p in phishing_terms:
    val = val + row[p + "_scount"] * 2
  for p in phishing_terms:
    val = val + row[p + "_count"]
    if row[p + '_count'] > 0:
      source = source + 1
  values.append(val)
  sources.append(source)
data['val'] = values
data['sources'] = sources

In [14]:
data.to_csv("full_data.csv", index=False)

In [15]:
manual_train_values = 1200
train_data_temp = pd.read_csv("train.csv")
actual_values = [1 if s == 100 else 0 for s in train_data_temp[0:manual_train_values]['Actual']]

train_data = data[0:manual_train_values]
train_data = train_data.drop(['Id', 'EmailTo', 'CC', 'SenderName', 'SenderEmailAddress', 'Subject', 'Body'], axis=1)
test_data = data.drop(['Id', 'EmailTo', 'CC', 'SenderName', 'SenderEmailAddress', 'Subject', 'Body'], axis=1)

In [16]:
# attempt to use an external dataset
'''
train_data_temp_2 = pd.read_csv("spam_ham_dataset.csv")
train_data_temp_2['text'][0].split('\r\n')
train_data_temp_2['Subject'] = ['' if isnan(s) else s.split('\r\n')[0] for s in train_data_temp_2['text']]
train_data_temp_2['Body'] = ['' if isnan(s) else ' '.join(s.split('\r\n')[1:]) for s in train_data_temp_2['text']]

for p in phishing_terms:
  train_data_temp_2[p + '_count'] = [0 if isnan(s) else s.lower().count(p) for s in train_data_temp_2['Body']]
  train_data_temp_2[p + '_scount'] = [0 if isnan(s) else s.lower().count(p) for s in train_data_temp_2['Subject']]
train_data_temp_2['Microsoft Teams Meeting_count'] = [0 if isnan(s) else s.count('Microsoft Teams Meeting') for s in train_data_temp_2['Body']]
train_data_temp_2['reply_thread_count'] = [0 if isnan(s) else min(s.lower().count('from:'), s.lower().count('sent:'), s.lower().count('to:')) for s in train_data_temp_2['Body']]
train_data_temp_2['https://'] = [0 if isnan(s) else s.count('https://') for s in train_data_temp_2['Body']]
train_data_temp_2['Body_isnan'] = [1 if isnan(e) else 0 for e in train_data_temp_2['Body']]
for p in good_words:
  train_data_temp_2[p + '_count'] = [0 if isnan(s) else s.lower().count(p) for s in train_data_temp_2['Body']]

values = []
for index, row in train_data_temp_2.iterrows():
  val = 0
  for p in phishing_terms:
    val = val + row[p + "_scount"] * 2
  for p in phishing_terms:
    val = val + row[p + "_count"]

  values.append(val)
train_data_temp_2['val'] = values

train_data_2 = train_data_temp_2.drop(['idx', 'label', 'text', 'label_num', 'Subject', 'Body'], axis=1)
train_dfs = [train_data, train_data_2]
train_data = pd.concat(train_dfs, ignore_index=True)
actual_values = list(actual_values) + list(train_data_temp_2['label_num'])
'''


'\ntrain_data_temp_2 = pd.read_csv("spam_ham_dataset.csv")\ntrain_data_temp_2[\'text\'][0].split(\'\r\n\')\ntrain_data_temp_2[\'Subject\'] = [\'\' if isnan(s) else s.split(\'\r\n\')[0] for s in train_data_temp_2[\'text\']]\ntrain_data_temp_2[\'Body\'] = [\'\' if isnan(s) else \' \'.join(s.split(\'\r\n\')[1:]) for s in train_data_temp_2[\'text\']]\n\nfor p in phishing_terms:\n  train_data_temp_2[p + \'_count\'] = [0 if isnan(s) else s.lower().count(p) for s in train_data_temp_2[\'Body\']]\n  train_data_temp_2[p + \'_scount\'] = [0 if isnan(s) else s.lower().count(p) for s in train_data_temp_2[\'Subject\']]\ntrain_data_temp_2[\'Microsoft Teams Meeting_count\'] = [0 if isnan(s) else s.count(\'Microsoft Teams Meeting\') for s in train_data_temp_2[\'Body\']]\ntrain_data_temp_2[\'reply_thread_count\'] = [0 if isnan(s) else min(s.lower().count(\'from:\'), s.lower().count(\'sent:\'), s.lower().count(\'to:\')) for s in train_data_temp_2[\'Body\']]\ntrain_data_temp_2[\'https://\'] = [0 if isnan(

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


#clf = DecisionTreeClassifier(random_state=0).fit(train_complete, actual_values)
#clf = RandomForestClassifier(n_estimators=100).fit(train_data, actual_values)
clf = XGBClassifier().fit(train_data, actual_values)

C:\Users\jyban\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:06:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [18]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [22]:
predict = clf.predict_proba(test_data)
out_df = pd.DataFrame()
out_df['Id'] = data['Id']
out_df['Flag'] = [a[1] for a in predict]
out_df

,Id,Flag
0,1,0.000769
1,2,0.003298
2,3,0.000071
3,4,0.000120
4,5,0.000255
...,...,...
4893,4894,0.000045
4894,4895,0.009234
4895,4896,0.000547
4896,4897,0.001891


In [21]:
predict

array([[9.9923140e-01, 7.6858146e-04],
       [9.9670190e-01, 3.2980742e-03],
       [9.9992895e-01, 7.1022179e-05],
       ...,
       [9.9945301e-01, 5.4701662e-04],
       [9.9810892e-01, 1.8910646e-03],
       [9.9983448e-01, 1.6549237e-04]], dtype=float32)

In [ ]:
out_df[['Id', 'Flag']].to_csv("output.csv", index=False)